# <font face="Verdana" size=6 color='#6495ED'> ANÁLISE ESTATÍSTICA DE DADOS
<font face="Verdana" size=3 color='#40E0D0'> Profa. Larissa Driemeier
<center><img src='https://drive.google.com/uc?export=view&id=1nW_7p_LyFhbR0ipjSekPcAj6kDoyK73R' width="800"></center>

Este notebook faz parte da aula 09 do curso IAD-001.

# Markov e Markov Oculto
Na teoria das probabilidades, um modelo de Markov é um modelo estocástico usado para modelar sistemas que mudam aleatoriamente. Supõe-se que os estados futuros dependam apenas do estado atual, não dos eventos que ocorreram antes dele (ou seja, assume-se a propriedade Markov).

A aplicação do Modelo de Markov oculto (Hidden Markov Models) inclui aprendizado por reforço e reconhecimento de padrões temporais, como fala, escrita, reconhecimento de gestos, marcação de parte do discurso, acompanhamento de partituras, descargas parciais e bioinformática.


In [ ]:
import numpy as np
import pandas as pd
from random import seed
from random import random
import matplotlib.pyplot as plt
%matplotlib inline

## Modelo de Markov para gerar texto
Reveja a teoria aplicada aqui em suas notas de aula, quando usamos uma parte do poema *E agora, José?*, de Carlos Drummond de Andrade.

<center><img src='https://drive.google.com/uc?export=view&id=1o4eOPXwRSujVT2QDnWrVrDEB4SPgPLBy' width="400"></center>


In [ ]:
from google.colab import files

uploaded = files.upload()

#Aqui você fará upload dos arquivos (disponíveis no Moodle):
# IHaveADream_MartinLutherKingJr.txt
#OAstronautaDeMarmore_NenhumDeNos.txt
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
discurso = open('OAstronautaDeMarmore_NenhumDeNos.txt', encoding='utf8').read()
#display the data
print(discurso)

In [ ]:
# 'Split' o discurso em um conjunto de palavras individuais
palavras = discurso.split()
print(palavras)

In [ ]:
#Criando pares de palavras chave e as palavras seguintes
#Para economizar espaço, uses um objeto generator (vide aula do Thiago).
def make_pairs(palavras):
  for i in range(len(palavras) - 1):
    yield (palavras[i], palavras[i + 1])
pairs = make_pairs(palavras)

In [ ]:
#Em seguida, vamos inicializar um dicionário vazio para armazenar os pares de palavras.
palavra_dict = {}
#Caso a primeira palavra do par (palavra chave) já seja uma chave no dicionário, basta anexar a próxima
#palavra potencial à lista de palavras que seguem a palavra chave. Mas se a palavra não for uma chave,
#crie uma nova entrada no dicionário e atribua a chave igual à primeira palavra do par.
#Caso fique confuso, veja a teoria, quando usamos o poema "E agora, José?"
for palavra_1, palavra_2 in pairs:
  if palavra_1 in palavra_dict.keys():
    palavra_dict[palavra_1].append(palavra_2)
  else:
    palavra_dict[palavra_1] = [palavra_2]

In [ ]:
# Modelo de Markov
#Vamos pegar uma palavra aleatória para iniciar nosso texto
#(deverá se iniciar com letra maiúscula para não pegarmos palavra no meio do texto)
start_palavra = np.random.choice(palavras)
while start_palavra.islower():
  start_palavra = np.random.choice(palavras)
chain = [start_palavra]
print(chain)
#Número total de palavras que terá nosso discurso
n_palavras = 20
for i in range(n_palavras):
  chain.append(np.random.choice(palavra_dict[chain[-1]]))

In [ ]:
#Finalmente, vamos exibir o texto  gerado
# 'join' retorna a nossa cadeia de palavras 'chain' como uma string
print(' '.join(chain))

O texto pode não fazer muito sentido, mas serve para você entender, de maneira simples, como as cadeias de Markov podem ser usadas para gerar textos automaticamente.

Agora, vamos ver mais algumas aplicações das cadeias de Markov e como elas são usadas para resolver problemas do mundo real.

**Google PageRank**: toda a web pode ser vista como um modelo de Markov, onde cada página da web pode ser um estado e os links ou referências entre essas páginas podem ser vistos como transições com probabilidades. Então, basicamente, independentemente de qual página da Web você começa a navegar, a chance de chegar a uma determinada página X da Web é uma probabilidade fixa.

**Previsão de digitação de palavras em telefones celulares**: Sabe-se que as cadeias de Markov são usadas para prever as próximas palavras ou preenchimento automático e sugestões. O aplicativo irá adivinhar qual é a sua próxima palavra mais provável com base nas combinações de palavras que você digitou antes (por exemplo, *dia* depois de *bom*" ou *como vai* com base em *Oi*). Da mesma maneira, ao completar as palavras, o algoritmo se baseia nas palavras que você repete várias vezes e, quanto mais vezes já apareceu, maior a probabilidade de aparecer novamente.

**Gerador de texto**: as cadeias de Markov são mais comumente usadas para gerar textos fictícios ou produzir grandes ensaios e compilar discursos. Também são usadas nos geradores de nome.

### Para você aumentar seu conhecimento!

O [link](https://towardsdatascience.com/using-a-markov-chain-sentence-generator-in-python-to-generate-real-fake-news-e9c904e967e) apresenta um gerador de Fake News. O programa está completo, e muito bem explicado. Pode ser que você precise procurar alguns comandos na internet para entendê-los melhor, mas o texto do link explica em detalhes o que exatamente cada parte do programa faz.

Se você está familiarizado com programação, ou gosta muito de desafios, pode gerar notícias seguindo a programação em [Markov](https://filiph.github.io/markov/).

__Mas veja bem:__
usar o primeiro link já é desafio o suficiente para o nível que estamos. O segundo exige uma capacidade de programação compatível somente com aqueles que já tem uma base sólida de programação, vinda da graduação.

## Cadeia de Markov

A cadeia de Markov possui estados possíveis
\begin{equation}
 \begin{bmatrix}
\mathbf{q}_1 & \mathbf{q}_2 & ... & \mathbf{q}_n \\
\end{bmatrix}
\end{equation}

E a probabilidade de transição de estado $\mathbf{A}$ (matriz $n \times n$)
\begin{equation}
a_{ij}=P\left( \mathbf{S}_{t+1} | \mathbf{S}_t \right)
\end{equation}

Dado o estado inicial
$\mathbf{\pi}=\mathbf{S}_0=P\left(\mathbf{S}_{0_i}=\mathbf{s}_i\right)$

A transição do estado $\mathbf{S}_{t}$ para o estado $\mathbf{S}_{t+n}$ é dada por,
\begin{equation}
\mathbf{S}_{t+n} = \mathbf{S}_{t} \mathbf{A}^n
\end{equation}

Sabe-se que, para uma matriz de transição $\mathbf{A}$ constante, a cadeia de Markov leva a um estado estacionário $\bar{\mathbf{S}}$, sendo que:
\begin{equation}
\bar{\mathbf{S}} = \bar{\mathbf{S}}\mathbf{A}
\end{equation}
ou, ainda, tem-se a matriz de transição estabilizada, $\bar{\mathbf{A}}$, dada por,

\begin{equation}
\bar{\mathbf{S}} = \mathbf{S}_0 \bar{\mathbf{A}}= \mathbf{\pi} \bar{\mathbf{A}}
\end{equation}

A partir da equação da estacionariedade $\mathbf{A}^T\bar{\mathbf{S}} ^T = \bar{\mathbf{S}}^T$, podemos ver que ${\mathbf{S}}$ deve ser um autovetor de $\mathbf{A}^T$ com um autovalor $1$.

Então, encontramos $\bar{\mathbf{S}}$ da seguinte forma,  para uma matriz $\mathbf{A}$ $3 \times 3$,
\begin{equation}
\begin{vmatrix}
  &a_{11}-1 & a_{21} & a_{31}  \\
  &a_{12} & a_{22}-1& a_{32}\\
  &a_{13} & a_{23}& a_{33}-1\\
\end{vmatrix}=0
\end{equation}

Por exemplo, para uma matriz $\mathbf{A}$ $3 \times 3$, tem-se:

\begin{equation}
\begin{split}
  &\bar{s}_1 a_{11} + \bar{s}_2 a_{21} + \bar{s}_3 a_{31} &= \bar{s}_1 \\
  &\bar{s}_1 a_{12} + \bar{s}_2 a_{22} + \bar{s}_3 a_{32} &= \bar{s}_2 \\
  &\bar{s}_1 a_{13} + \bar{s}_2 a_{23} + \bar{s}_3 a_{33} &= \bar{s}_3 \\
\end{split}
\end{equation}
e
\begin{equation}
\bar{s}_1 + \bar{s}_2  + \bar{s}_3= 1
\end{equation}

Em python, `np.linalg.solve` resolve um sistema de equações lineares. E usaremos as equações acima para gerar um sistema:

\begin{equation}
\begin{split}
  &\bar{s}_1 \left(a_{11}-1\right) &+ \bar{s}_2 a_{21} &+ \bar{s}_3 a_{31}   &= 0 \\
  &\bar{s}_1 a_{12} &+ \bar{s}_2 \left(a_{22}-1\right) &+ \bar{s}_3 a_{32}   &= 0 \\
  &\bar{s}_1  a_{13} &+ \bar{s}_2 a_{23} &+ \bar{s}_3 \left(a_{33}-1\right)   &= 0 \\
  &\bar{s}_1       &+ \bar{s}_2 &+\bar{s}_3 &= 1
  \end{split}
\end{equation}

A função `np.linalg.solve` resolve $Ax=b$ para matriz $A$ quadrada, então aplicamos a transformação em ambos os lados $A^T Ax = A^T b$.

Depois, podemos encontrar a matriz de transição estabilizada $\bar{\mathbf{A}}$, utilizando-se a equação $\bar{\mathbf{S}}^T =  \bar{\mathbf{A}}^T\mathbf{\pi}^T$.

Pode-se porvar que a matriz será,sempre,

$$
\bar{\mathbf{A}} =\begin{bmatrix}
\mathbf{\bar S}\\
\mathbf{\bar S}\\
\vdots
\end{bmatrix}
$$

onde $\pi=[\pi_1 \quad \pi_2 \quad \pi_3 \cdots]$ é a matriz de probabilidades inicial.


[Brinque](https://setosa.io/ev/markov-chains/) com cadeia de Markov.

### Exemplo de aplicação

<center><img src='https://drive.google.com/uc?export=view&id=140mitXL21DSCXh5uNuQkx6bnxzpdD1Yh' width="300"></center>

Para $\pi =[3/5 \quad 2/5]$ e matriz de transição dada por,

$$
\mathbf{A} = \begin{bmatrix}
0.5 & 0.5  \\
0.25 & 0.75
\end{bmatrix}
$$

Queremos resolver:

$$
\mathbf{\bar{S}} \mathbf{A} = \mathbf{\bar{S}}
\Rightarrow \mathbf{A} \mathbf{\bar{S}}^T =\mathbf{\bar{S}}^T
$$

Ou seja, resolver $(\mathbf{A}^T - \mathbf{I}) \mathbf{\bar{S}}^T = 0$

$$
\mathbf{A}^T = \begin{bmatrix}
0.5 & 0.25 \\
0.5 & 0.75
\end{bmatrix}, \quad
\mathbf{A}^T - \mathbf{I} = \begin{bmatrix}
-0.5 & 0.25 \\
0.5 & -0.25
\end{bmatrix}
$$

Resolvendo o sistema:

$$
-0.5 \bar{s}_1 + 0.25 \bar{s}_2 = 0 \Rightarrow \bar{s}_1 = 0.5 \bar{s}_2
$$

Mas,

$$
\bar{s}_1 + \bar{s}_2 = 1 \Rightarrow 0.5 \bar{s}_2 + \bar{s}_2 = 1 \Rightarrow \frac{3}{2} \bar{s}_2 = 1 \Rightarrow \bar{s}_2 = \frac{2}{3}, \quad \bar{s}_1 = \frac{1}{3}
$$

Portanto:

$$
\boldsymbol{\bar{S}}^T = \begin{bmatrix}
\frac{1}{3} \\
\frac{2}{3}
\end{bmatrix}
$$

A matriz estabilizada:

$$
\boldsymbol{\bar{S}} = \begin{bmatrix}
\frac{1}{3} &
\frac{2}{3}
\end{bmatrix}, \quad
\bar{\mathbf{A}} = \begin{bmatrix}
\frac{1}{3} & \frac{2}{3} \\
\frac{1}{3} & \frac{2}{3}
\end{bmatrix}
$$


Em python, usaremos `np.linalg.solve` para resolver o sistema de equações lineares.

In [ ]:
#Resposta:
A = np.array([[0.5,0.5],[0.25,0.75]])
A2 = np.append(A.T-np.identity(2),[[1,1]],axis=0)
b = (np.array([0,0,1])).T
#matriz deixa de ser quadrada
sbar = np.linalg.solve(A2.T.dot(A2), A2.T.dot(b))
print('A distribuição estável é dada por:\n',sbar)

w,v=np.linalg.eig(A.T) # autovetores são as colunas de v
for i,autovalor in enumerate(w):
  if autovalor == 1:
    autovetor = v[:,i]
    sum = np.sum(autovetor)
    autovetor = autovetor/sum
sbar = autovetor
print('A distribuição estável é dada por:\n',sbar)

In [ ]:
Abar = np.array([sbar,sbar])
print('A matriz de transformação estável é dada por:\n',Abar)

In [ ]:
S0=np.array([3/5,2/5])
S0.dot(Abar)

### Exemplo Marcas de fast food

<center><img src='https://drive.google.com/uc?export=view&id=1chbEE-Oa_9PHMILDPh2Ex5HdCHa7kN00' width="400"></center>

Queremos analisar a transação de clientes em uma área de alimentação de um shopping. Analisamos os clientes almoçando nos três lugares mostrados. A probabilidade do cliente voltar no dia seguinte ou ir para outro lugar é definida conforme a matriz de transição
\begin{equation}
\mathbf{A} = \begin{bmatrix}
0.8 & 0.1 & 0.1 \\
0.2 & 0.7 & 0.1 \\
0.1 & 0.3 & 0.6 \\
\end{bmatrix}
\end{equation}

1. Dados os estados iniciais $S_0=\left[0.4 \quad 0.24 \quad 0.36 \right]$, defina a distribuição provável de 500 clientes no dia seguinte.
1. Plote o gráfico das variáveis em função das iterações.
2. Encontre a distribuição estável $\bar{\mathbf{s}}$.
1. Encontre a matriz de transição estável, $\bar{\mathbf{A}}$.
2. Encontre $\bar{\mathbf{s}}$ para diferentes entradas $S_0$ usando $\bar{\mathbf{A}}$ obtida no item anterior.





In [ ]:
A = np.array([[0.8, 0.1, 0.1],
              [0.2, 0.7, 0.1],
              [0.1, 0.3, 0.6]])
state=np.array([[0.4, 0.24, 0.36]])
stateHist=state
dfStateHist=pd.DataFrame(state)
distr_hist = [[0,0,0]]
plt.figure()
for x in range(20):
  state=np.dot(state,A)
  stateHist=np.append(stateHist,state,axis=0)
  dfDistrHist = pd.DataFrame(stateHist)
dfDistrHist.plot()
#Item 1
value_500 = 500*dfDistrHist.iloc[0]
print('1. O número de pessoas que irão aos locais 0,1,2 serão:',value_500)
#Item 2
print('Item 2')
plt.xlabel('Número de iterações', fontsize = 12)
plt.ylabel('$X_0,X_1, X_2$', fontsize = 12)
plt.legend()
plt.show()

In [ ]:
# Faremos um sistema linear para resolver com np.linalg.solve
#Para isso precisamos unir todas as equações conforme explicado no texto
A2 = np.append(A.T-np.identity(3),[[1,1,1]],axis=0)
b = (np.array([0,0,0,1])).T
#matriz deixa de ser quadrada usar teorema de Rouché-Capelli
xbar = np.linalg.solve(A2.T.dot(A2), A2.T.dot(b))
#Item 3
print('3. A distribuição estável é dada por:',xbar)
#Item 4
Abar = np.array([xbar,xbar,xbar])
print('4. A matriz de transformação estável é dada por:',Abar)
#Item 5
#Estado inicial 1:
print('5. Independente do estado inicial, o estado estacionário é o mesmo:')
print(state.dot(Abar))
state2=np.array([[0.1, 0.5, 0.4]])
print(state2.dot(Abar))
state3=np.array([[0.3, 0.5, 0.2]])
print(state2.dot(Abar))

### Exemplo da Loja de Doces
Temos duas lojas de doces, $A$ e $B$. Sabe-se,a través de pesquisas, que $30\%$ dos clientes da loja $A$ vão para loja $B$ todo mês, e $20\%$ dos clientes da loja $B$ vão para loja $A$ todo mês. Qual a matriz de distribuição estabilizada $\bar{\mathbf{A}}$ e, supondo que tenhamos 200 clientes ao total, como se distribuirão no estado estacionário?

In [ ]:
 #Sua resposta


In [ ]:
#Resposta:
A = np.array([[0.7,0.3],[0.2,0.8]])
A2 = np.append(A.T-np.identity(2),[[1,1]],axis=0)
b = (np.array([0,0,1])).T
xbar = np.linalg.solve(A2.T.dot(A2), A2.T.dot(b))
print('A distribuição estável é dada por:',xbar)
Abar = np.array([xbar,xbar,xbar])
print('A matriz de transformação estável é dada por:',Abar)
print('Para 200 clientes ao total, tem-se a seguinte distribuição nas lojas A e B, respectivamente,'
       ,int(round(xbar[0]*200)),'e',int(round(xbar[1]*200)))

### Exemplo de análise de lealdade à marca
Três marcas $A,B,C$ foram analisadas e verificou-se que:


*   Na marca $A$: $10\%$ dos clientes se movem para $B$ e $10\%$ se movem para $C$
*   Na marca $B$: $20\%$ dos clientes se movem para $A$ e $20\%$ se movem para $C$
*   Na marca $C$: $10\%$ dos clientes se movem para $A$ e $20\%$ se movem para $B$

Encontre a matriz de distribuição estabilizada e o vetor de estabilização.



In [ ]:
#Resposta: Você deve encontrar xbar = [8/19, 5/19, 6/19]

### Cadeias de Markov com estados absorventes
Um estado da cadeia de Markov é chamado de *estado absorvente* se $a_{ii} = 1 $ e, por consequência, qualquer valor da linha $i$, $a_{ij}, i \neq j$, vale $0$. Uma cadeia de Markov é dita absorvente se existe ao menos um estado absorvente, ou se for possível, a partir de qualquer estado, atingir um estado absorvente, não necessariamente em um único passo.

Veja a figura abaixo. Veja que a matriz da figura é definida como:
\begin{equation}
\mathbf{A} = \begin{bmatrix}
1.0 & 0.0 & 0.0 \\
0.2 & 0.7 & 0.1 \\
0.3 & 0.1 & 0.6 \\
\end{bmatrix}
\end{equation}

Dessa forma,
\begin{equation}
\mathbf{A}^2 = \begin{bmatrix}
1.0 & 0.0 & 0.0 \\
0.37 & 0.50 & 0.13 \\
0.50 & 0.13 & 0.37 \\
\end{bmatrix}
\end{equation}

\begin{equation}
\mathbf{A}^3 = \begin{bmatrix}
1.0 & 0.0 & 0.0 \\
0.509 & 0.363 & 0.128 \\
0.637 & 0.128 & 0.235 \\
\end{bmatrix}
\end{equation}

Se formos adiante, perceberemos que a primeira coluna os valores crescem, se aproximando de 1, a primeira linha se mantém constante, e os demais valores estão indo para zero. Veja, por exemplo,

\begin{equation}
\mathbf{A}^5 = \begin{bmatrix}
1.0 & 0.0 & 0.0 \\
0.707 & 0.198 & 0.095 \\
0.802 & 0.095 & 0.103 \\
\end{bmatrix}
\end{equation}

Eventualmente, o estado estacionário leva a:
\begin{equation}
\bar{\mathbf{A}} = \begin{bmatrix}
1 & 0 & 0 \\
1 & 0 & 0 \\
1 & 0 & 0 \\
\end{bmatrix}
\end{equation}
isto é,
\begin{equation}
\bar{s} = \begin{bmatrix}
1 & 0 & 0
\end{bmatrix}
\end{equation}

Isto é, o estado $\mathbf{\mathcal{\bar P}} $ absorve todo o universo analisado.

Foi usada a palavra *eventualmente* no texto acima. Veja o exemplo de um estudo realizado pela Universidade da Carolina do Norte em pacientes de um determinado hospital. O problema foi modelado por uma cadeia de Markov: $0$ (morto), $1$ (estado desfavorável), $2 $(estado favorável). A matriz de transição tem um ciclo de 72 horas,

\begin{equation}
\mathbf{A} = \begin{bmatrix}
1.0 & 0.0 & 0.0 \\
0.085 & 0.779 & 0.136 \\
0.017 & 0.017 & 0.966 \\
\end{bmatrix}
\end{equation}

Perceba que $a_{11} = 1$, ou seja, $a_1$ é um estado de absorção, uma vez que o paciente morto, a cada passo ele continuará morto. Os estados $2$ e $3$ são
os estados de transição, e a partir de qualquer um destes é possível chegar no estado de absorção. Daí, a cadeia é absorvente. Porém, não é sensato acreditar que todos os pacientes acabarão mortos. Então, a matriz de transformação estabilizada é diferente daquela mostrada anteriormente.

Um [estudo](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1828134/) realizado pela universidade de medicina do Texas em pacientes em UTI de um determinado hospital foi modelado por uma cadeia de Markov, com 8 estados possíveis após o primeiro dia de internação na unidade de terapia intensiva foram avaliados para cada paciente de UTI: enfermaria do mesmo hospital (W); Enfermaria de outro hospital, não UTI ou estar em outro hospital após a alta da UTI para outro hospital (B); UTI de outro hospital (O); Unidade de alta dependência ou área de atendimento intermediário mesmo hospital (U); mesma UTI ou outra UTI dentro do mesmo hospital (I); outro local mesmo hospital (H), saída (E) e óbito (D). Cada destino identifica um estado na cadeia de Markov. **Leia o artigo e veja uma aplicação clara do modelo.**

#### Exercício

Para as duas matrizes abaixo,

\begin{equation}
\mathbf{A} = \begin{bmatrix}
1.0 & 0.0 &  0.0 & 0.0 \\
0.0 & 1.0 &  0.0 & 0.0 \\
0.2 & 0.2 &  0.5 & 0.1 \\
0.3 & 0.1 &  0.2 & 0.4 \\
\end{bmatrix}
\end{equation}


\begin{equation}
\mathbf{A} = \begin{bmatrix}
1.0 & 0.0 & 0.0 \\
0.4 & 0.3 & 0.3 \\
0.3 & 0.2 & 0.5 \\
\end{bmatrix}
\end{equation}

1.   Encontre a matriz estabilizada de transição de estado;
2.   Use diferentes valores de $\mathbf{S}_0$ e verifique os valores encontrados. Qual sua conclusão?

### Markov oculto


#### <font color='green'>Seu trabalho </font>

<center><img src='https://drive.google.com/uc?export=view&id=1r_BcKlp6AVj46yeQlT9fNxZY1qeajrH_' width="200"></center>

<font color='green'>
A aula de hoje, apesar de complexa, foi apenas uma introdução aos modelos de Markov, que serão muito úteis em nossas próximas etapas. Na lição de casa, você resolverá um problema mais elaborado, com até 3 colegas. A nota desta lição, substituirá sua menor nota.
</font>

<font color='green'>
Suponha que alguém lance uma moeda dez vezes e forneça o resultado:. Porém, o lançador tem duas moedas e muda aleatoriamente de uma moeda para outra. Uma moeda é equilibrada $E$ e a outra é tendenciosa $NE$, que resulta em cara 80% das vezes que é lançada. Após cada lançamento, o jogador pode manter a mesma moeda (probabilidade 0,8), mudar para a outra moeda (0,20). Além disso, a probabilidade de começar com a moeda equilibrada é 0,8. Com essas informações,

1. Monte o problema e as matrizes;
2. Qual a probabilidade da sequência $CKKCCCCCKC$, onde $C$ é cara e $K$ é coroa, ocorrer?
3. Qual a sequência mais provável das moedas utilizadas?
4. Qual a probabilidade dessa sequência ocorrer cem vezes? Estude o fenômeno de underflow e proponha a solução.

</font>